# Creation of GDP and CPI Mimicking portfolios

## Data Prep

In [2]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv('merged_data.csv', index_col=0)

In [7]:
data['def'] = np.log(data['corp_bond_return']/data['corp_bond_return'].shift(1)) - data['20y_return']
data['term'] = 

,BAA,AAA,10 year,1 year,corp_bond_return,BBKMGDP,CPIAUCSL,20y_return,rf,30y_return,90day,mkt,ISM,SL,SM,SH,BL,HM,BH
1962-01-31,5.08,4.42,4.10,3.29,NaN,5.031371,30.040,-0.001602,0.002348,0.004844,0.002496,-0.036147,60.9,-2.6050,-0.8629,2.2592,-5.0016,-1.9027,0.3934
1962-02-28,5.07,4.42,4.00,3.21,NaN,9.228796,30.110,0.007828,0.001989,0.003634,0.002243,0.019512,61.1,0.7054,0.7314,1.7967,1.8606,2.4060,2.4047
1962-03-31,5.04,4.39,3.86,2.97,NaN,4.330766,30.170,0.017130,0.002007,0.026249,0.002244,-0.004688,60.6,0.4216,-1.1191,-0.5598,-0.3751,-0.2782,-2.1517
1962-04-30,5.02,4.33,3.86,3.07,NaN,2.061945,30.210,0.019020,0.002278,0.016849,0.002475,-0.063435,55.1,-6.3111,-7.2753,-6.8347,-6.6656,-5.5658,-6.0788
1962-05-31,5.00,4.28,3.90,2.99,NaN,2.304132,30.240,0.010481,0.002296,-0.004814,0.002465,-0.084620,52.2,-12.2520,-10.3050,-9.5501,-9.0826,-6.7701,-6.2353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,6.07,4.90,3.68,4.74,3986.08,-0.144042,298.598,0.071208,0.002856,0.077931,0.003114,0.058064,49.0,-0.4429,3.0537,2.6164,5.1322,5.4544,4.8403
2022-12-31,5.59,4.43,3.88,4.73,3947.76,-1.266861,298.990,-0.025772,0.003379,-0.035953,0.003280,-0.040356,48.4,-5.8863,-6.7824,-5.4729,-7.1720,-3.9805,-4.9431
2023-01-31,5.50,4.40,3.52,4.68,4230.53,NaN,300.536,NaN,NaN,NaN,NaN,NaN,47.4,14.0186,11.0995,8.1850,7.4981,5.5366,5.1740
2023-02-28,5.59,4.56,3.92,5.02,4005.19,NaN,301.648,NaN,NaN,NaN,NaN,NaN,47.7,NaN,NaN,NaN,NaN,NaN,NaN
